In [1]:
# imports
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import asyncpg

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# using datetime module
import datetime;

In [2]:
async def fetch_data(conn):
    # Execute the SQL query to fetch data from your table
    sql_query = "SELECT name, company_domain FROM recruiters"
    rows = await conn.fetch(sql_query)

    # Separate lists for each column
    hiring_manager_list = []
    company_domain_list = []

    # Iterate through the rows and store data in separate lists
    for row in rows:
        hiring_manager_list.append(row['name'])
        company_domain_list.append(row['company_domain'])
        
    print(f"{hiring_manager_list}\n{company_domain_list}")
    return hiring_manager_list, company_domain_list

In [3]:
async def post_data(conn, manager_email, hiring_manger, company_domain):
    insert_query = "UPDATE recruiters SET email = $1, first_email = $4 WHERE name=$2 AND company_domain=$3;"
    # ct stores current time
    ct = datetime.datetime.now()
    await conn.execute(insert_query, manager_email, hiring_manger, company_domain, ct)


In [4]:
def send_email(sender_email, sender_password, recipient_email, subject, body):
    # Create a MIMEText object
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the body to the message
    msg.attach(MIMEText(body, 'plain'))

    # Connect to the SMTP server (Gmail's SMTP server in this case)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

In [5]:
def login(driver):
    driver.get("https://app.apollo.io/#/login")
    element = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, '/html/body/div[2]/div/div[2]/div[2]/div/div/div/div[3]/div[2]/a/button'))
                    )

    # wait for manual login to finish
    input("Login and Press Enter to continue: ")

In [6]:
def search(driver, hiring_manager, company_domain):
    driver.get("https://app.apollo.io/#/people")
    time.sleep(2)


    company_drobdown_btn = driver.find_element(
        By.XPATH, "//div[contains(@class, 'zp_YfgQq')]/span[text()='Company']/parent::div")
    print(company_drobdown_btn)
    # driver.execute_script("arguments[0].scrollIntoView(true);", company_drobdown_btn)
    company_drobdown_btn.click()
    
    company_input_box = driver.find_element(By.CLASS_NAME, "Select-input")
    company_input_box.send_keys(company_domain)
    
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable(
            (By.CLASS_NAME, "Select-option"))
    )
    driver.find_element(By.CLASS_NAME, "Select-option").click()
    time.sleep(1)

    search_input_box = driver.find_element(
        By.XPATH, "//input[@placeholder='Search']")
    search_input_box.send_keys(hiring_manager)
    search_input_box.send_keys(Keys.RETURN)

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH,
                                            "//button//span[text()='Access email']"))
        )

        hiring_manager_element = driver.find_element(By.XPATH, f"//a[text()='{hiring_manager}']")
        print(hiring_manager_element)

        driver.execute_script("arguments[0].scrollIntoView(true);", hiring_manager_element)
        manager_url = driver.find_element(By.XPATH, f"//a[text()='{hiring_manager}']").get_attribute("href")

        print(manager_url)
    
        # Open a new window 
        driver.execute_script("window.open('');") 
        driver.switch_to.window(driver.window_handles[1])


        driver.get(manager_url)
        time.sleep(5)

        # First try to get the email directly if it's already visible
        try:
            # Check if the email is already visible (meaning button was already clicked)
            manager_email = driver.find_element(By.CSS_SELECTOR, "//a[contains(text(), '@')]").text
            print("Email already accessible: ", manager_email)
        except:
            # Email not directly visible, need to click the button
            try:
                # Find and click the access email button
                button_element = driver.find_element(By.XPATH, "//button//span[text()='Access email']")
                print("Found access email button, clicking it")
                button_element.click()
                time.sleep(3.5)
                
                # Now try to get the email after clicking
                try:
                    manager_email = driver.find_element(By.XPATH, "//a[contains(text(), '@')]").text
                    print("Retrieved email after clicking button: ", manager_email)
                except:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    print(f"COULD NOT FIND EMAIL OF: {hiring_manager} ({company_domain}) even after clicking button\n")
                    return -1
            except:
                # Could not find the access email button
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(f"COULD NOT FIND ACCESS BUTTON FOR: {hiring_manager} ({company_domain})\n")
                return -1
            
        print("manager_email: ", manager_email)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        return manager_email


        # driver.get(manager_url)
        # time.sleep(5)

        # try:
        #     button_element = driver.find_element(By.XPATH, "//button//span[text()='Access email']")
        #     manager_email = button_element.text
        # except:
        #     access_email_btn = driver.find_element(By.CSS_SELECTOR, "#general_information_card > div > div > div.zp_w0Cv0.zp_EXndU > div.zp_TNVkG.zp_f9CwO > div > div > div > div.zp_hpMiB.zp_FAqrS > button.zp-button.zp_zUY3r.zp_n9QPr.zp_rhXT_")
        #     access_email_btn.click()
        #     time.sleep(3.5)
        #     try:
        #         manager_email = driver.find_element(By.CSS_SELECTOR, ".zp-link.zp_OotKe.zp_dAPkM.zp_Iu6Pf").text
        #     except:
        #         driver.close()
        #         driver.switch_to.window(driver.window_handles[0])
        #         print(f"COULD NOT FIND EMAIL OF: {hiring_manager} ({company_domain})\n")
        #         return -1
        # print("manager_email: ", manager_email)
        # driver.close()
        # driver.switch_to.window(driver.window_handles[0])
        # return manager_email
    except:
        print(f"COULD NOT FIND EMAIL OF: {hiring_manager} ({company_domain})\n")
        return -1

In [40]:
def clear_search_filters(driver, name_len):
    WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable(
                            (By.CSS_SELECTOR, '.zp_bWS5y.zp_XnbMe'))
                    )
    company_input_box = driver.find_element(By.CSS_SELECTOR, '.zp_bWS5y.zp_XnbMe')
    for i in range(0,name_len):
        company_input_box.send_keys(Keys.BACKSPACE)
    company_filter_cross = driver.find_element(By.CSS_SELECTOR, "#main-app > div.zp_GhGgo > div > div > div.zp_HrbxN > div.zp_nOEP5 > div > div > div > div > div > div > div > div.zp_qvvEC.zp_SS_4T > div > div.zp_RmCxU.finder-sidebar-filters.zp_OMnUn > div.zp-accordion.zp_BRLTI.zp__jgHx.zp_BOvnT > div.zp-accordion-header.zp_LPwjF.zp_BRLTI.zp_FCU6A > span > button > i.zp-icon.mdi.mdi-close.zp_dZ0gM.zp_j49HX.zp_uAV5p")
    company_filter_cross.click()
    company_drobdown_btn = driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/div[2]/div/div[2]/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[4]/div[5]/div")
    company_drobdown_btn.click()
    

In [38]:
#selenium initializations
chrome_options = Options()
# driver = webdriver.Chrome()
# chrome_options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

# change according to database configurations
# conn = await asyncpg.connect(user="postgres", password="1234", host="127.0.0.1", port=5432, database="postgres")

# Example usage
# Generate app password here -> https://myaccount.google.com/apppasswords
sender_password = 'USE APP PASSWORD HERE'

subject = 'Test Email'
body = 'This is a test email sent from Python.'

driver = webdriver.Chrome(options=chrome_options)

login(driver)

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001049a3b78 cxxbridge1$str$ptr + 2778912
1   chromedriver                        0x000000010499c1b0 cxxbridge1$str$ptr + 2747736
2   chromedriver                        0x00000001044f1e24 cxxbridge1$string$len + 92932
3   chromedriver                        0x0000000104539158 cxxbridge1$string$len + 384568
4   chromedriver                        0x000000010457a500 cxxbridge1$string$len + 651744
5   chromedriver                        0x000000010452d2e4 cxxbridge1$string$len + 335812
6   chromedriver                        0x0000000104969d04 cxxbridge1$str$ptr + 2541740
7   chromedriver                        0x000000010496cfc8 cxxbridge1$str$ptr + 2554736
8   chromedriver                        0x000000010494aa44 cxxbridge1$str$ptr + 2414060
9   chromedriver                        0x000000010496d828 cxxbridge1$str$ptr + 2556880
10  chromedriver                        0x000000010493b998 cxxbridge1$str

In [41]:
from supabase import Client, create_client
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv(override=True)

# Initialize Supabase client
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

async def fetch_data():
    # Get recruiters with company and job information
    response = supabase.rpc('get_recruiters_with_job_info', params={}).execute()
    rows = response.data

    # Group jobs by recruiter to handle multiple postings
    recruiter_jobs = {}
    for row in rows:
        recruiter_id = row['id']
        if recruiter_id not in recruiter_jobs:
            recruiter_jobs[recruiter_id] = {
                'name': row['name'],
                'company_domain': row['company_domain'],
                'company_name': row['company_name'],
                'first_email': row.get('first_email'),
                'last_email': row.get('last_email'),
                'email_count': row.get('email_count', 0),
                'jobs': []
            }
        recruiter_jobs[recruiter_id]['jobs'].append({
            'job_title': row['job_title'],
            'job_id': row['job_id']
        })

    hiring_manager_list = []
    company_domain_list = []
    email_history = []
    job_info_list = []

    # Process each recruiter only once, combining job information if needed
    for recruiter_id, data in recruiter_jobs.items():
        # if should_send_email(data.get('last_email'), data.get('email_count', 0)):
        hiring_manager_list.append(data['name'])
        company_domain_list.append(data['company_domain'])

        # Combine job information for the email
        job_info_list.append({
            'company_name': data['company_name'],
            'jobs': data['jobs'],  # Now we pass all jobs
            # Use first job as primary for tracking
            'primary_job_id': data['jobs'][0]['job_id']
        })

        email_history.append({
            'recruiter_id': recruiter_id,
            'first_email': data.get('first_email'),
            'last_email': data.get('last_email'),
            'email_count': data.get('email_count', 0)
        })

    print(f"Found {len(hiring_manager_list)} recruiters to contact")
    return hiring_manager_list, company_domain_list, email_history, job_info_list

In [49]:
hiring_manager_list, company_domain_list, email_history, job_info_list = await fetch_data()
print(hiring_manager_list[0])
print(company_domain_list[0])
print(email_history[0])
print(job_info_list[0])


Found 242 recruiters to contact
Jazmin Dalisay
https://www.telusdigital.com/careers
{'recruiter_id': 411, 'first_email': None, 'last_email': None, 'email_count': 0}
{'company_name': '', 'jobs': [{'job_title': 'Remote - Data Analyst (Thai speakers) ', 'job_id': 4178483740}, {'job_title': 'Homebased - Online Data Analyst in India (Gujarati speakers) ', 'job_id': 4178482314}, {'job_title': 'Homebased - Personalized Internet Assessor in Kyrgyzstan (Kyrgyz speakers) ', 'job_id': 4178485553}, {'job_title': 'WFH - Online Data Analyst in India (Malayalam speakers) ', 'job_id': 4178485045}, {'job_title': 'Homebased - Online Data Analyst in India (Malayalam speakers) ', 'job_id': 4178480844}], 'primary_job_id': 4178483740}


In [57]:

# for i in range(2, 4):
#     manager_email = search(
#         driver, hiring_manager_list[i], company_domain_list[i])
#     print(manager_email)
    # clear_search_filters(driver, len(hiring_manager_list[i]))

import random
index = random.randint(0, len(hiring_manager_list) - 1)

print(hiring_manager_list[index])
print(company_domain_list[index])
print(index)
# manager_email = search(driver, hiring_manager_list[index], company_domain_list[index])
# print(manager_email)




Yogesh S’ profile
http://www.winfort.net/
174


### Below code is for testing

In [ ]:
Yogesh S’ profile
http://www.winfort.net/
174

In [ ]:
# hiring_manager_list, company_domain_list = await fetch_data(conn)

for i in range(3, len(hiring_manager_list)):
    manager_email = search(driver, hiring_manager_list[i], company_domain_list[i])
    if(manager_email != -1):
        pass
    #    await post_data(conn, manager_email, hiring_manager_list[i], company_domain_list[i])
    clear_search_filters(driver, len(hiring_manager_list[i]))

driver.close()

await post_data(conn, "theo.constantinides@synbiotix.com", "Theo Constantinides", "http://www.synbiotix.com/")